# cesm Rainfall Anomalies

In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
import utils.timePeriod as tp

In [2]:
import xarray
import numpy
import cftime
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [3]:
secondsPerDay = 60*60*24
secondsToTimeP = secondsPerDay*1000 #seconds per day, convert m to mm
#secondsToTimeP = secondsPerDay*_index.nDaysOfInterest["precAnom"]*1000/7 #seconds per month

# Calculate a climatology

Based on the control run, calculate monthly anomalies

In [5]:
preclDa = fh.loadModelData('CESM-LME', 'PRECL', _model.cesmCntl)
preccDa = fh.loadModelData('CESM-LME', 'PRECC', _model.cesmCntl)


E:/CMIP5-PMIP3/CESM-LME/mon/PRECL/
Files imported: 
 ['E:/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECL.085001-184912.nc', 'E:/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECL.185001-200512.nc']
E:/CMIP5-PMIP3/CESM-LME/mon/PRECC/
Files imported: 
 ['E:/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECC.085001-184912.nc', 'E:/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECC.185001-200512.nc']


In [6]:
prectDa = (preclDa.PRECL+preccDa.PRECC)
prectDa.name='prect'

In [7]:
monMeansDa=prectDa.groupby('time.month').mean(dim='time')

In [8]:
monMeansDa.where(
        (monMeansDa.lat>-50) & (monMeansDa.lat<0) & (monMeansDa.lon>100) & (monMeansDa.lon<170),
        drop=True
    ).load()

<xarray.DataArray 'prect' (month: 12, lat: 26, lon: 27)>
array([[[3.33550858e-08, 3.30955281e-08, 3.28213048e-08, ...,
         3.86643180e-08, 4.18805683e-08, 3.88527432e-08],
        [3.00628074e-08, 2.95105629e-08, 2.89366842e-08, ...,
         3.78427885e-08, 4.48913582e-08, 4.56386715e-08],
        [2.45112126e-08, 2.35937776e-08, 2.25785524e-08, ...,
         3.51952032e-08, 4.28528821e-08, 5.62610900e-08],
        ...,
        [1.24986315e-07, 1.06377158e-07, 1.21130313e-07, ...,
         5.72019303e-08, 5.38809779e-08, 5.01305415e-08],
        [8.87420697e-08, 8.41698267e-08, 1.29954486e-07, ...,
         4.03243234e-08, 3.73808753e-08, 3.43152351e-08],
        [7.96472150e-08, 1.23047855e-07, 1.35169927e-07, ...,
         3.67590332e-08, 3.41318334e-08, 3.14309787e-08]],

       [[3.41190649e-08, 3.39739685e-08, 3.38874564e-08, ...,
         3.98528854e-08, 4.32115002e-08, 3.98622682e-08],
        [2.92681257e-08, 2.89401942e-08, 2.87052231e-08, ...,
         3.83719438e-08, 4.54697577e-08, 4.53024036e-08],
        [2.33549073e-08, 2.27472281e-08, 2.20327827e-08, ...,
         3.53170186e-08, 4.25204512e-08, 5.47665095e-08],
...
        [1.06703894e-07, 9.93074920e-08, 9.82102293e-08, ...,
         5.25900710e-08, 4.99475128e-08, 4.73066564e-08],
        [9.55758921e-08, 8.19404420e-08, 1.07734351e-07, ...,
         3.48941285e-08, 3.21528368e-08, 2.96335489e-08],
        [7.86909311e-08, 1.04142735e-07, 1.06253260e-07, ...,
         2.72927849e-08, 2.49968810e-08, 2.27232793e-08]],

       [[3.08103267e-08, 3.03643191e-08, 2.99856246e-08, ...,
         3.59287071e-08, 3.92442132e-08, 3.65880517e-08],
        [2.85066424e-08, 2.78317032e-08, 2.70605760e-08, ...,
         3.60445007e-08, 4.31427587e-08, 4.29995914e-08],
        [2.41862264e-08, 2.31146906e-08, 2.19890204e-08, ...,
         3.46644669e-08, 4.24424762e-08, 5.50420225e-08],
        ...,
        [1.14976785e-07, 1.02066338e-07, 1.15361040e-07, ...,
         4.86838054e-08, 4.68219028e-08, 4.47055761e-08],
        [9.13112856e-08, 7.94980579e-08, 1.21394294e-07, ...,
         3.28403544e-08, 3.07965067e-08, 2.87164106e-08],
        [7.80295863e-08, 1.13973307e-07, 1.22832219e-07, ...,
         2.79913319e-08, 2.59534438e-08, 2.38161171e-08]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -48.32 -46.42 -44.53 -42.63 ... -4.737 -2.842 -0.9474
  * lon      (lon) float64 102.5 105.0 107.5 110.0 ... 160.0 162.5 165.0 167.5
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

# Carve out an area of interest, and calculate anomalies

In [10]:

for experiment in [*_model.cesmFullForcings, _model.cesmCntl]:
    
    #load it
    preclDa = fh.loadModelData('CESM-LME', 'PRECL', experiment)
    preccDa = fh.loadModelData('CESM-LME', 'PRECC', experiment)
    prectDa = (preclDa.PRECL+preccDa.PRECC)
    prectDa.name='prect'
    
    #anom
    anomDa=prectDa.groupby('time.month')-monMeansDa
    anomDa.name='precAnom'
        
    #grab area around Australia
    domainDa=anomDa.where(
        (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
        drop=True
    )
    
    # convert to mm/year and store
    domainDa.to_netcdf('results/cesmMonthlyPrec/'+experiment+'.nc')
    
     #rcp85 extensions
    if experiment in ['002','003', '008', '009']: 
        
        #extensions
        preclDa = fh.loadModelData('CESM-LME', 'PRECL', 'LME.'+experiment)
        preccDa = fh.loadModelData('CESM-LME', 'PRECC', 'LME.'+experiment)
        extPrectDa = (preclDa.PRECL+preccDa.PRECC)
        extPrectDa.name='prect'
        
        prectDa = xarray.concat([prectDa, extPrectDa], 'time')


        #anom
        anomDa=prectDa.groupby('time.month')-monMeansDa
        anomDa.name='precAnom'

        #grab area around Australia
        domainDa=anomDa.where(
            (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
            drop=True
        )

        #  store
        domainDa.to_netcdf('results/cesmMonthlyPrec/LME.'+experiment+'.nc')
        
    
    

E:/CMIP5-PMIP3/CESM-LME/mon/PRECL/
Files imported: 
 ['E:/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECL.085001-184912.nc', 'E:/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECL.185001-200512.nc']
E:/CMIP5-PMIP3/CESM-LME/mon/PRECC/
Files imported: 
 ['E:/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECC.085001-184912.nc', 'E:/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECC.185001-200512.nc']


C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


# Time averaging for the anomalies

In [4]:
monthlyAnomDa=xarray.concat([
    xarray.open_dataset('results/cesmMonthlyPrec/'+iExp+'.nc') for iExp in _model.cesmAll],
    'experiment')
    

monthlyAnomDa=monthlyAnomDa.assign({'experiment':_model.cesmAll})


warmSeasonAnomDa=secondsToTimeP*tp.averageForTimePeriod(monthlyAnomDa)
warmSeasonAnomDa = warmSeasonAnomDa.assign_attrs([*warmSeasonAnomDa.attrs, ('units','mm/month'), ('timePeriod','Warm Season')])

In [5]:
warmSeasonAnomDa.to_netcdf('results/cesmPrecAnoms.nc')

# Do that all again for Ts

In [6]:
tsDa = fh.loadModelData('CESM-LME', 'TS', _model.cesmCntl)

monMeansDa=tsDa.TS.groupby('time.month').mean(dim='time')

monMeansDa.where(
        (monMeansDa.lat>-50) & (monMeansDa.lat<0) & (monMeansDa.lon>100) & (monMeansDa.lon<170),
        drop=True
    ).load()

E:/CMIP5-PMIP3/CESM-LME/mon/TS/
Files imported: 
 ['E:/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.TS.085001-184912.nc', 'E:/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.TS.185001-200512.nc']


<xarray.DataArray 'TS' (month: 12, lat: 26, lon: 27)>
array([[[282.3029 , 282.37982, 282.453  , ..., 285.12256, 285.29666,
         284.7596 ],
        [283.459  , 283.4972 , 283.5798 , ..., 286.4485 , 286.78424,
         286.2769 ],
        [284.62888, 284.65228, 284.7156 , ..., 287.77362, 288.0664 ,
         287.87802],
        ...,
        [300.6783 , 297.80142, 301.59268, ..., 302.07162, 302.01572,
         301.93628],
        [297.66537, 296.9991 , 301.1753 , ..., 301.60736, 301.49063,
         301.36142],
        [296.50525, 301.20428, 301.6382 , ..., 301.0947 , 300.9362 ,
         300.7725 ]],

       [[282.7254 , 282.80463, 282.88074, ..., 285.83698, 286.00516,
         285.44794],
        [283.92322, 283.95853, 284.0398 , ..., 287.20044, 287.55966,
         286.83868],
        [285.1558 , 285.1732 , 285.2386 , ..., 288.56027, 288.86633,
         288.47354],
...
        [300.4305 , 298.107  , 302.19376, ..., 301.91733, 301.91766,
         301.88547],
        [297.88455, 297.2016 , 301.8522 , ..., 301.48654, 301.40683,
         301.33057],
        [296.87653, 302.21036, 302.53326, ..., 301.00912, 300.89197,
         300.77475]],

       [[281.152  , 281.22144, 281.2946 , ..., 283.62625, 283.7778 ,
         283.2848 ],
        [282.34995, 282.37405, 282.44168, ..., 284.8917 , 285.18594,
         284.7441 ],
        [283.42715, 283.44675, 283.4926 , ..., 286.09814, 286.3729 ,
         286.20593],
        ...,
        [300.74658, 297.96945, 302.2252 , ..., 302.01318, 301.9762 ,
         301.9234 ],
        [297.82126, 297.0872 , 301.7551 , ..., 301.58798, 301.49747,
         301.39532],
        [296.7144 , 301.92868, 302.28696, ..., 301.1211 , 300.98816,
         300.84598]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -48.32 -46.42 -44.53 -42.63 ... -4.737 -2.842 -0.9474
  * lon      (lon) float64 102.5 105.0 107.5 110.0 ... 160.0 162.5 165.0 167.5
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

# Carve out an area of interest, and calculate anomalies

In [7]:

for experiment in [*_model.cesmFullForcings, _model.cesmCntl]: #
    
    #load it
    tsDa = fh.loadModelData('CESM-LME', 'TS', experiment).TS
    
    #anom
    anomDa=tsDa.groupby('time.month')-monMeansDa
    anomDa.name='tsAnom'
        
    #grab area around Australia
    domainDa=anomDa.where(
        (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
        drop=True
    )
    
    # convert to mm/year and store
    domainDa.to_netcdf('results/cesmMonthlyTs/'+experiment+'.nc')
    
     #rcp85 extensions
    if experiment in ['002','003', '008', '009']: 
        
        #extensions
        extTsDa = fh.loadModelData('CESM-LME', 'TS', 'LME.'+experiment)
        tsDa=xarray.concat([tsDa, extTsDa.TS], 'time')

        #anom
        anomDa=tsDa.groupby('time.month')-monMeansDa
        anomDa.name='tsAnom'

        #grab area around Australia
        domainDa=anomDa.where(
            (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
            drop=True
        )

        domainDa.to_netcdf('results/cesmMonthlyTs/LME.'+experiment+'.nc')
        
    
    

E:/CMIP5-PMIP3/CESM-LME/mon/TS/
Files imported: 
 ['E:/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.TS.085001-184912.nc', 'E:/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.TS.185001-200512.nc']


C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


# Time averaging for the anomalies

In [8]:
monthlyAnomDa=xarray.concat([
    xarray.open_dataset('results/cesmMonthlyTs/'+iExp+'.nc') for iExp in _model.cesmAll],
    'experiment')
    

monthlyAnomDa=monthlyAnomDa.assign({'experiment':_model.cesmAll})

warmSeasonAnomDa=tp.averageForTimePeriod(monthlyAnomDa)

warmSeasonAnomDa.to_netcdf('results/cesmTsAnoms.nc')